In [1]:
import numpy as np
import pandas as pd
import re
from spatial import *
import time

Point in polygon test
Point 1 is within the polygon
Point 2 is outside the polygon
R-tree test
Query result:
[1, 3]


In [2]:
# 311 data
df = pd.read_csv('311_join/311_Service_Requests_2014.csv')

/Users/Daynan/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (8,17,18,21,38,40,41,42,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# spatial data
dfs = pd.read_csv('data_for_analysis_map/all_nyc_sa_all_features.csv')

In [4]:
dfc = df.copy()
dfc = dfc[['Unique Key','Created Date','Agency','Complaint Type','Descriptor',
           'Location Type', 'Incident Zip', 'Incident Address','Latitude','Longitude','Location']]

In [5]:
# label calls about heat
def heat(data):
    if (data=='Air Conditioning Problem') | (data=='HEAT') | (data=='HEAT RELATED') | (data=='AIR-CONDITIONER'):
        return True
    else:
        return False

dfc['heat'] = dfc.Descriptor.map(lambda x: heat(x))

In [6]:
# reformat point
def ptconvert(data):
    try:
        data = data.strip('()').split(', ')
        return Point(float(data[0]),float(data[1]))
    except:
        return Point(0,0)

dfc['point'] = dfc.Location.map(lambda x: ptconvert(str(x)))

In [7]:
def polyconvert(data):
    try:
        allpoints = data.strip('MULTIPOLYGON ()')
        allpoints = allpoints.strip('()').split(', ')
        new_pts = []
        for i in allpoints:
            i = i.split(' ')
            x = float(i[1])
            y = float(i[0])
            pt = x,y
            pt = Point(x,y)
            new_pts.append(pt)
        return Polygon(new_pts)
    except:
        return 'nan'

In [8]:
dfs['poly'] = dfs.geometry.map(lambda x: polyconvert(x))

In [9]:
# create rtree index on all calls
start_time = time.time()
all_call_rtree = index.Index()
for i,p in enumerate(dfc['point']):
    x = p.x
    y = p.y
    all_call_rtree.insert(i,(x,y,x,y))
print("--- %s seconds ---" % (time.time() - start_time))

--- 127.338639975 seconds ---


In [10]:
# create rtree index on heat calls
start_time = time.time()
heat_rtree = index.Index()
dfh = dfc[dfc.heat].reset_index()
for i,p in enumerate(dfh['point']):
    x = p.x
    y = p.y
    heat_rtree.insert(i,(x,y,x,y))
print("--- %s seconds ---" % (time.time() - start_time))

--- 6.44579982758 seconds ---


In [11]:
# rtree polygon query
def rquery(rindx, poly,heat=False):
    try:
        # obtain the polygon bound
        x_min = float('inf')
        x_max = -float('inf')
        y_min = float('inf')
        y_max = -float('inf')
        for i in poly.points:
            if i.x < x_min:
                x_min = i.x
            if i.x > x_max:
                x_max = i.x
            if i.y < y_min:
                y_min = i.y
            if i.y > y_max:
                y_max = i.y

        # query on the bound
        bounded = list(rindx.intersection((x_min,y_min,x_max,y_max)))
        
        if heat:
            df = dfh
        else:
            df = dfc
        
        count = 0
        for i in bounded:
            pt = df.ix[i][-1]
            if poly.contains(pt):
                count += 1
        
#        return len(bounded)
        return count
    except AttributeError:
        return 0

In [12]:
start_time = time.time()
dfs['all_calls'] = dfs.poly.map(lambda x: rquery(all_call_rtree,x))
print("--- %s seconds ---" % (time.time() - start_time))

--- 288.992995977 seconds ---


In [13]:
start_time = time.time()
dfs['heat_calls'] = dfs.poly.map(lambda x: rquery(heat_rtree,x,heat=True))
print("--- %s seconds ---" % (time.time() - start_time))

--- 16.9029350281 seconds ---


In [14]:
dfs['heat_ratio'] = dfs.heat_calls*1.0/dfs.all_calls

In [15]:
dfs.to_csv('all_nyc_all_features_311.csv',index=False)

In [27]:
# dfs_for_map = dfs[['FIPS','geometry']]
# df_for_map = df[['Unique Key','Descriptor','Location']]

In [29]:
# dfs_for_map.to_csv('polys.csv',index=False)
# df_for_map.to_csv('calls.csv',index=False)